In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.estimators import model_fn

In [2]:
# read in some of the data
df = pd.read_csv('../data/facial keypoints training.csv', nrows=5000)

In [3]:
df = df[['left_eye_center_x', 'left_eye_center_y','Image']]
# for now only train on the left eye center
df = df[df['left_eye_center_x'].notnull() & df['left_eye_center_y'].notnull()]
# round the left eye x, y coordinates to integers (could multiply by powers of ten to get higher position resolution)
df[['left_eye_center_x', 'left_eye_center_y']] = df[['left_eye_center_x', 'left_eye_center_y']].round(0)
print(df[['left_eye_center_x', 'left_eye_center_y']].head())

   left_eye_center_x  left_eye_center_y
0               66.0               39.0
1               64.0               35.0
2               65.0               35.0
3               65.0               37.0
4               67.0               40.0


In [4]:
# encode. must give column names if numeric
left_eye_x = pd.get_dummies(df['left_eye_center_x'], columns=['left_eye_center_x'])
left_eye_y = pd.get_dummies(df['left_eye_center_y'], columns=['left_eye_center_y'])
# convert faces to dataframe
faces = df.Image.apply(lambda x: pd.Series(np.fromstring(x, dtype=np.float32, sep=' ')))

In [5]:
# see the results
print(faces.head())
print(faces.shape)

    0      1      2      3      4      5      6      7      8      9     \
0  238.0  236.0  237.0  238.0  240.0  240.0  239.0  241.0  241.0  243.0   
1  219.0  215.0  204.0  196.0  204.0  211.0  212.0  200.0  180.0  168.0   
2  144.0  142.0  159.0  180.0  188.0  188.0  184.0  180.0  167.0  132.0   
3  193.0  192.0  193.0  194.0  194.0  194.0  193.0  192.0  168.0  111.0   
4  147.0  148.0  160.0  196.0  215.0  214.0  216.0  217.0  219.0  220.0   

   ...    9206  9207  9208  9209  9210  9211  9212   9213   9214   9215  
0  ...    33.0  29.0  30.0  34.0  39.0  49.0  62.0   70.0   75.0   90.0  
1  ...     1.0   1.0   1.0   1.0   1.0   1.0   1.0    1.0    1.0    1.0  
2  ...    64.0  60.0  56.0  61.0  70.0  69.0  71.0   78.0   78.0   77.0  
3  ...     1.0   1.0   1.0   1.0   1.0   1.0   1.0    1.0    1.0    1.0  
4  ...    33.0  34.0  37.0  37.0  43.0  46.0  83.0  140.0  170.0  176.0  

[5 rows x 9216 columns]
(4990, 9216)


In [6]:
# free space
del df['Image']
faces = faces.as_matrix()
left_eye_x = left_eye_x.as_matrix()
num_labels = left_eye_x.shape[1]
image_size = 96

# have a look
print(faces.shape)
print(left_eye_x.shape)

(4990, 9216)
(4990, 48)


In [7]:
def model(features, targets, mode, params):
    '''model function for estimator'''
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
        
    # loss
    logits = tf.matmul(features, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=logits))
    
    # predictions
    train_prediction = tf.nn.softmax(logits)

    # saver
    saver = tf.train.Saver(tf.global_variables())
    
    #predictions_dict = {"x-coordinate": logits.argmax(axis=1)}
    predictions_dict = {"x-coordinate": logits}
    
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=params['learning_rate'],
        optimizer='SGD')

    return model_fn.ModelFnOps(
        mode=mode,
        predictions=predictions_dict,
        loss=loss,
        train_op=train_op)

In [16]:
nn = tf.contrib.learn.Estimator(model_fn=model, params={'learning_rate':0.5})

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_num_ps_replicas': 0, '_model_dir': None, '_task_type': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe71fc25160>, '_environment': 'local', '_evaluation_master': '', '_save_summary_steps': 100, '_num_worker_replicas': 0, '_master': '', '_save_checkpoints_secs': 600, '_is_chief': True}


In [17]:
nn.fit(x=faces, y=left_eye_x, steps=100)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Create CheckpointSaverHook.


/home/boot/.local/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmptm6kvpfk/model.ckpt.
INFO:tensorflow:step = 1, loss = 23007.4
INFO:tensorflow:Saving checkpoints for 20 into /tmp/tmptm6kvpfk/model.ckpt.
INFO:tensorflow:Loss for final step: 2.94713e+07.


Estimator(params={'learning_rate': 0.5})

In [18]:
nn.evaluate(x=faces, y=left_eye_x, steps=1)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Starting evaluation at 2017-05-13-12:45:07
INFO:tensorflow:Restoring parameters from /tmp/tmptm6kvpfk/model.ckpt-20


/home/boot/.local/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-05-13-12:45:08
INFO:tensorflow:Saving dict for global step 20: global_step = 20, loss = 2.84236e+07


{'global_step': 20, 'loss': 28423558.0}

In [19]:
predictions = nn.predict(x=faces[:10,:], as_iterable=True)
for i, p in enumerate(predictions):
  print("Prediction %s: %s" % (i + 1, p['x-coordinate'].argmax()))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /tmp/tmptm6kvpfk/model.ckpt-20


/home/boot/.local/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


Prediction 1: 32
Prediction 2: 32
Prediction 3: 32
Prediction 4: 32
Prediction 5: 32
Prediction 6: 32
Prediction 7: 32
Prediction 8: 32
Prediction 9: 32
Prediction 10: 32
